Custom atmospheric profiles
===========================

.. admonition:: Overview

   This tutorials intoduces Eradiate's capabilities to handle customized atmospheric profiles beyond classical rescaling.

.. admonition:: Prerequisites

   The reader should familiarize themselves with the user guide about Eradiate terminology and the `one-dimensional experiment features`. Additionally users should be familiar with the `molecular atmosphere basics` tutorial.

.. admonition:: What you will learn

   This tutorial will teach you how to format a custom atmospheric profile for Eradiate and how to use it in a `MolecularAtmosphere`.

.. admonition:: Additional requirements

   An example atmosphere profile is available at `<PATH>` in the `eradiate-data` repository

----

## Standard profiles and rescaling

First, let us refresh the basics of standard atmosphere profiles and classic rescaling in Eradiate.

Eradiate uses the `joseki` package for handling atmosphere profiles.

We import a few necessary packages and create two molecular atmospheres. One with a basic profile and one with a rescaled molecule.

In [ ]:
%load_ext eradiate
import eradiate
import joseki
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
from eradiate.scenes.atmosphere import MolecularAtmosphere
from eradiate import unit_registry as ureg

In [ ]:
eradiate.set_mode("ckd")

In [ ]:
us_standard = MolecularAtmosphere(
    thermoprops={
        "identifier": "afgl_1986-us_standard",
        "z": np.linspace(0.0, 120.0, 121) * ureg.km,
    },
)

If you recall the `molecular atmosphere basics` tutorial, you will recognize this workflow. We have now created a molecular atmosphere, using the AFGL 1986 US-Standard atmosphere profile.

Next, let us create another atmosphere with a rescaled amount of carbon dioxide.

In [ ]:
# First, create the original standard profile:
thermoprops = joseki.make(
    identifier="afgl_1986-us_standard",
    z=np.linspace(0.0, 120.0, 121) * ureg.km,
)

# Then, rescale it:
rescaled = thermoprops.joseki.rescale(
    factors={
        "O3": 1.25,
    }
)

# Finally instanciate the MolecularAtmosphere class with the rescaled
# thermophysical profile
molecular_rescaled = MolecularAtmosphere(
    thermoprops=rescaled
)

Now we can plot the vertical profile of ozone and compare both atmospheres.

In [ ]:
us_standard._thermoprops.x_O3.plot(y="z", label="US-Standard")
molecular_rescaled._thermoprops.x_O3.plot(y="z", label="Rescaled to 125%")
plt.legend()

## Custom profiles

The fundamental limitation of classic rescaling is that it only controls the total integrated concentration of an atmospheric constituent. The detailed vertical distribution can not be changed. Eradiate adresses this problem by allowing users to add fully custom distributions of constituent gasses in the `MolecularAtmosphere` object.

Essentially, the requirements for a custom atmosphere profile are the same as for the `thermoprops` object, we defined above:

In [ ]:
thermoprops

- Thermophysical properties must have only one dimension, named "z", which describes the vertical grid. This vertical grid must have regular spacing. In the example, level range from 0 to 120 kilometers and are 1 kilometer apart
- The data variables p, t, and n, which represent the atmospheric pressure, temperature and air number density, must be present to define the vertical profile of the atmosphere.
- The molar fractions of all gases which are available in Eradiate's absorption data (data variables whose names start with `x_`) should be present, but in its default configuration, Eradiate will not raise an Error if a molecule is missing.

**The absorption by molecules which are not present in the thermophysical properties dataset will not be taken into account in the simulations!**

Now, let us load a custom profile. 

In [ ]:
thermoprops_custom = xr.load_dataset("/home/schunkes/src/eradiate-data/tutorials/custom_atmosphere/custom_profile.nc")

In [ ]:
thermoprops_custom

You will notice, this profile does not follow the requirements listed above. If you would try to use this profile to create a molecular atmosphere for Eradiate, you would find that it fails. The profile we just loaded contains three extra dimensions: `time`, `lon`, and `lat`. We remove them by squeezing the profile.

In [ ]:
thermoprops_custom = thermoprops_custom.squeeze()
thermoprops_custom

To create an Eradiate atmosphere with this custom profile, we use the same approach as for the classic rescaling.

In [ ]:
molecular_custom = MolecularAtmosphere(
    thermoprops=thermoprops_custom
)

This profile is a modification of the AFGL-1986 US-Standard profile. Notably it contains a vertical profile for ozone, based on data from the EAC4 dataset.

Adding this new profile to the previous plot reveals the difference.

In [ ]:
us_standard._thermoprops.x_O3.plot(y="z", label="US-Standard")
molecular_rescaled._thermoprops.x_O3.plot(y="z", label="Rescaled to 125%")
molecular_custom._thermoprops.x_O3.plot(y="z", label="Custom profile")
plt.legend()
plt.title("")
plt.show()

As you can see, the peak of the distribution is positioned lower in the atmosphere, compared to the standard profile. Note the discontinuity around 55km. The ozone data in EAC4 only covers altitudes up to circa 55km. Above that, we stitched this data to the original data of the US-Standard atmosphere. At 55km elevation, atmospheric pressure and in turn number density and absorption are so low, that this discontinuity has negligible effect.

## Running a simulation

Finally, to gauge the effect of the custom atmospheric profile, let us set up a simulation of a spectral band that is affected by absorption by ozone. We choose Band 6 of the OLCI instrument on the Sentinel 3A satellite.

In [ ]:
def get_exp(atmosphere):
    surface = eradiate.scenes.bsdfs.LambertianBSDF(reflectance=0.3)
    illumination = eradiate.scenes.illumination.DirectionalIllumination(zenith=45, azimuth=0)
    measure = eradiate.scenes.measure.MultiDistantMeasure.hplane(
        zeniths=np.arange(-60, 61, 2),
        azimuth=0,
        srf="sentinel_3a-olci-6",
    )

    exp = eradiate.experiments.AtmosphereExperiment(
        surface=surface,
        illumination=illumination,
        measures=measure,
        atmosphere=atmosphere
    )
    return exp

In [ ]:
results = {}
for name, atmo in {"USST": us_standard,"custom": molecular_custom}.items():
    exp = get_exp(atmo)
    results[name] = eradiate.run(exp, spp=1000)

In [ ]:
fig, ax = plt.subplots(1,1)
for name, result in results.items():
    result.brf_srf.plot(ax=ax, label=name, x="vza")
plt.legend(title="Atmosphere type")

We see, that the custom atmosphere profile shows significantly different absorption than the US-Standard atmosphere.

----

Final words
-----------

You have learned about the difference between standard atmosphere profiles, classic rescaling and customized thermophysical properties profiles. You have learned how to format a dataset to use it in an Eradiate Molecular atmosphere.
Finally you have learned how to use a custom profile in an Eradiate simulation.

Further reading
---------------

*TBD*